## SheffleNet v1
- Paper : (2017)_ShuffleNet-An Extremely Efficient Convolutional Neural Network for Mobile Devices
- 배경
    - 모바일, 임베디드 환경에 네트워크 적용을 위한 모델 크기를 줄이거나 성능을 유지하는 네트워크
    - mobileNet에서 1x1 convolution이 대부분의 연산량을 차지하며 이러한 부분을 개선 목적

- 이전 MobileNet에서의 핵심이 Depthwise separable convolution이였다면, ShuffleNet은 그것 이외에 2가지의 스킬을 가짐
    - (Pointwise) Group Convolution
    - Channel Shuffle
<hr>


### ShuffleNet 요약
- Channel Shuffle : 1x1 pointwise group conv의 group 별 정보교환이 없는 문제점을 해결하기 위해 각 그룹의 Output을 중간단계에서 shuffle 및 group끼리 교환
- (ResNet) : 1x1 -> 3x3 -> 1x1 의 bottleneck 구조 
- (ResNetXt) : 1x1 gconv  -> channel shuffle -> 3x3conv -> 1x1 gconv
- (shufflenet : stride2) 1x1conv -> channel shuffle -> 3x3 conv -> 1x1gconv -> concat 
- 일반적으로 Group 나눴을 때 더 결과가 좋으며, 더 작은 모델일수록 group=1일때에 비해, group이 늘어날때의 성능 차이가 커짐 -> 작은 모델일 수록 channel 수가 중요하다는 것


## ShuffleNet 핵심 아이디어
- **1. 1x1 Pointwise Group convolution**
    - group convolution : Alexnet이 gpu 메모리 한계로 conv의 모든 채널을 한번에 학습할 수 없어 나눠서 학습한 것에서 착안 -> **AlexNet이 높게 나온건 해당시기엔 GPU때문이였지만 이런 Group화도 있다 라고 생각**
    - 단 그룹별로 conv 진행시, group끼리의 cross talk(정보 교환)이 없기 때문에 전부 따로 학습하게 되는 부작용 존재.  ==> 피쳐의 Representation 약화 ==> Channel Shuffle 사용

- **2. Channel Shuffle **
    - 1x1 point wise group conv의 group 별 정보교환이 없는 문제점을 해결하기 위한 방안
    - **각 그룹의 Output을 중간단계에서 shuffle 및 group끼리 교환 shuffle**

![shufflenet_shuffle](img/shufflenet_shuffle.PNG)


<hr>

## ShuffleNet unit
- (a) ResNet - Channel shuffle이 없는 unit
- (b) ResNetXt - Channel shuffle 추가한 것
- (c) ShuffleNet - Output 크기가 절반으로 줄어드는 연산을 하고 있기 때문에, AVG Pool을 통해 Shortcut path의 크기를 조절 및 Channel concat을 통한 channel dimension 증가

![shufflenet_unit](img/shufflenet_unit.PNG)

- input size = c * h * w , bottleneck channel = m
- ResNet
    - 첫 1x1 conv = hwcm / 마지막단 1x1 conv도 hwmc => 2hwmc
    - 중간 3x3 conv = h x w x m => bottleneck이 m 연산량은 9 hwmm^2

- ShuffleNet
    - group별 연산이므로 2hwmc/g + 9mhw (3x3 DWConv) => hw(2cm/g + 9m)
    
<table>
    <thead><tr><td>ResNet</td><td>ResNeXt</td><td>ShuffleNet</td></tr></thead>
    <tr><td>hw(2cm + 9m^2)</td><td>hw(2cm + 9m^2 / g)</td><td>hw(2cm/g + 9m)</td></tr>
</table>

<hr>


## Group 수 별 실험결과
- Complexity가 모두 비슷하게 맞춰져있음
- Group의 수를 늘리면서 얻어진 이득(연산량 감소)에 따라 그만큼 Channel 수를 늘린 것)

![shufflenet_architecture](img/shufflenet_architecture.PNG)


<hr>

## ShuffleNet 추가실험 - Complexity & error
- ShuffleNet의 파라미터 갯수를 Scale로 주어가며 실험
- 일반적으로 Group 나눴을 때 더 결과가 좋음
- **더 작은 모델일수록 group=1일때와, group이 늘어날때의 성능 차이가 커짐**
    - -> **작은 모델일 수록 channel 수가 중요하다는 것**
    
![shufflenet_complexity](img/shufflenet_complexity.PNG)

<hr>

## Shufflenet vs Mobilenet

- MobileNet과 Complexity를 비슷하게 맞추어놓고 실험시 더 좋은 성능을 가짐

![shufflenet_vs_mobilenet](img/shufflenet_vs_mobilenet.png)

## ShuffleNet - InferenceTime

![shufflenet_result_inferencetime](img/shufflenet_result_inferencetime.PNG)

## 참고문헌
- https://masterzone.tistory.com/45
- https://hichoe95.tistory.com/54
